Based on: https://medium.com/howtoai/video-classification-with-cnn-rnn-and-pytorch-abe2f9ee031

In [4]:
import os

path2data = "./data"
listOfFields = os.listdir(path2data)
listOfFields, len(listOfFields)

(['1471', '1112', '1461', '1470', '1464', '1463'], 6)

In [1]:
import random

import cv2
from matplotlib import pyplot as plt

import albumentations as A

In [2]:
def visualize(image):
    plt.figure(figsize=(10, 10))
    plt.axis('off')
    plt.imshow(image)

In [6]:
listOfFields[0]

'1471'